# PACKAGES

In [ ]:
import pandas as pd
import pickle as pkl
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import chi2_contingency

# IMPORT STREAMED DATA 

In [ ]:
## stream the data, bit by bit (it is precombined, a combo of faker data + test data)

# collect from a kaftka producer 

# google api 

# watoomi data - 1000 rows of each per 30 seconds 
# TstIdTr.pkl
# TstIdTr_synth.pkl


# PREPROCESSING 

In [ ]:
# Categorical features
categorical_features = [
    'ProductCD',
    'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
    'addr1', 'addr2',
    'P_emaildomain', 'R_emaildomain',
    'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
    'DeviceType', 'DeviceInfo',
    'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 
    'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 
    'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32', 
    'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38'
]

# Numerical features (excluding TransactionID and isFraud)
numerical_features = [
    col for col in trIdTr.columns if col not in categorical_features + ['TransactionID', 'isFraud']

       # categorical

## label encoding for all categorical features 
## set all categories as categorical pandas format

# # Label Encoding for all categorical features
# for col in categorical_features:
#     if trIdTr[col].dtype == 'object' or col in categorical_features:
#         le = LabelEncoder()
#         trIdTr[col] = le.fit_transform(trIdTr[col].astype(str))


# Assuming label encoders for categorical features were saved during training
for col in categorical_features:
    le = LabelEncoder()
    # Fit encoder on training data and apply to test data
    le.fit(trIdTr[col])  # Assuming `trIdTr[col]` is from the training set
    test_data[col] = le.transform(test_data[col].astype(str))


In [ ]:
# Don't perform target encoding during testing; use the target-encoded features stored during training.

# Assuming target encoding was done on training set, apply the same encoding to test data
for col in high_cardinality_features:
    test_data[f'{col}_target_enc'] = test_data[col].map(target_encoding_map)


In [ ]:



 

# Setting all as categorical pandas format
for col in categorical_features:
    trIdTr[col] = trIdTr[col].astype('category')
]

## identify cardinality of categorical features 
## do target encoding for high cardinality categoricals
## add into dataframe 

# Calculate cardinality for categorical features
cardinality = {col: trIdTr[col].nunique() for col in categorical_features}
print("Cardinality of categorical features:", cardinality)

# Define a threshold for high cardinality

## HERE WE SHOULD MANUALLY INCLUDE CHANGES
high_cardinality_threshold = 20  # Features with more than 20 unique categories are considered high cardinality
low_cardinality_features = [col for col, unique_vals in cardinality.items() if unique_vals <= high_cardinality_threshold]
high_cardinality_features = [col for col, unique_vals in cardinality.items() if unique_vals > high_cardinality_threshold]

# Set low cardinality features as 'category' dtype (LightGBM will handle them natively)
for col in low_cardinality_features:
    trIdTr[col] = trIdTr[col].astype('category')

# Perform target encoding for high-cardinality features
# Ensure 'isFraud' is present
assert 'isFraud' in trIdTr.columns, "The 'isFraud' target column is missing."

for col in high_cardinality_features:
    # Initialize target encoded column
    trIdTr[f'{col}_target_enc'] = np.nan
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for train_idx, val_idx in skf.split(trIdTr, trIdTr['isFraud']):
        train_data, val_data = trIdTr.iloc[train_idx], trIdTr.iloc[val_idx]
        
        # Compute target encoding (mean of 'isFraud' for each category)
        means = train_data.groupby(col)['isFraud'].mean()
        
        # Map the target encoding to the validation set
        trIdTr.loc[val_idx, f'{col}_target_enc'] = val_data[col].map(means)
    
    # For any remaining NaN values, we leave them as NaN (LightGBM can handle them)
    # No need to fill NaNs since LightGBM handles them naturally

print("Target encoding added for high-cardinality features:", high_cardinality_features)

# trIdTr.to_csv('./processed_datasets/trIdTr1.csv')
# trIdTr = pd.read_csv('./processed_datasets/trIdTr1.csv')

# feature engineering 

### add in TransactionDT enriched columns:

### numerical 
trIdTr['TransactionDT_days'] = trIdTr['TransactionDT'] // (24 * 60 * 60)  # Convert to days 

### categorical

trIdTr['TransactionDT_weekday'] = trIdTr['TransactionDT_days'] % 7  # Day of the week (0-6)
trIdTr['TransactionDT_hour'] = (trIdTr['TransactionDT']//3600) % 24  # Calculate hour of the day (0-23)

### numerical
trIdTr['TransactionDT_hours'] = (trIdTr['TransactionDT'] // (60 * 60)) % 24  # Extract hours - hours since reference point in dataset

trIdTr['hour_sin'] = np.sin(2 * np.pi * trIdTr['TransactionDT_hour'] / 24)
trIdTr['hour_cos'] = np.cos(2 * np.pi * trIdTr['TransactionDT_hour'] / 24)
trIdTr['weekday_sin'] = np.sin(2 * np.pi * trIdTr['TransactionDT_weekday'] / 7)
trIdTr['weekday_cos'] = np.cos(2 * np.pi * trIdTr['TransactionDT_weekday'] / 7)

####    TransactionAmt split into whole dollars and cents (so one dollars/whole number, and one cents column)
trIdTr['TransactionAmt_dollars'] = trIdTr['TransactionAmt'] // 1  # Extract whole dollars
trIdTr['TransactionAmt_cents'] = (trIdTr['TransactionAmt'] * 100) % 100  # Extract cents

# update the feature sets: 

### categorical_features
### numerical_features , 

# Update categorical_features with newly created categorical columns
categorical_features += [
    'TransactionDT_weekday', 'TransactionDT_hour',  # New categorical features
]

# Update numerical_features with newly created numerical columns
numerical_features += [
    'TransactionDT_days', 'TransactionDT_hours',  # Existing numerical features
    'hour_sin', 'hour_cos', 'weekday_sin', 'weekday_cos',  # Cyclical features
    'TransactionAmt_dollars', 'TransactionAmt_cents',  # New features from TransactionAmt
    'DeviceInfo_target_enc', 'id_30_target_enc', 'id_25_target_enc', 'id_31_target_enc', 
    'id_20_target_enc', 'addr2_target_enc', 'card5_target_enc', 'card2_target_enc', 
    'addr1_target_enc', 'id_13_target_enc', 'id_19_target_enc', 'id_21_target_enc', 
    'card3_target_enc', 'id_17_target_enc', 'P_emaildomain_target_enc', 'id_33_target_enc', 
    'id_22_target_enc', 'id_26_target_enc', 'R_emaildomain_target_enc', 'card1_target_enc',
    'DeviceInfo_target_enc', 'id_30_target_enc', 'id_25_target_enc', 'id_31_target_enc', 
    'id_20_target_enc', 'addr2_target_enc', 'card5_target_enc', 'card2_target_enc', 
    'addr1_target_enc', 'id_13_target_enc', 'id_19_target_enc', 'id_21_target_enc', 
    'card3_target_enc', 'id_17_target_enc', 'P_emaildomain_target_enc', 'id_33_target_enc', 
    'id_22_target_enc', 'id_26_target_enc', 'R_emaildomain_target_enc', 'card1_target_enc', 
    'id_14_target_enc'
]


# Check the final lists
print("Updated categorical features:", categorical_features)
print("Updated numerical features:", numerical_features)

### note TransactionDT_hours,TransactionDT_days, are not important features so should not be put into either numieracl or categorerial (but still remain in dataset)

### find set which are not in either

# Get all columns from the dataframe
all_columns = set(trIdTr.columns)

# Combine categorical and numerical features
all_features = set(categorical_features + numerical_features)

# Identify features that are not in either categorical or numerical lists
missing_features = all_columns - all_features

# Print the missing features
print("Features not included in either categorical or numerical features:", missing_features)

# remove unnecessary columns 

# remove unnecessary columns - go through 
### categorical_features: any features with target_enc in numerical_features set
### numerical_features: remove TransactionDT_hours, TransactionDT_days

# check that all columns are in categorical_features or numerical_features; if not, remove column, print removed column names. 
# exclusions are: 'Unnamed: 0', 'isFraud', 'TransactionID'

# Remove target-encoded features from categorical_features if they exist in numerical_features
categorical_features = [col for col in categorical_features if col + '_target_enc' not in numerical_features]

# Remove TransactionDT_hours and TransactionDT_days from numerical_features
numerical_features = [col for col in numerical_features if col not in ['TransactionDT_hours', 'TransactionDT_days']]


# Create a set of all columns that should be included
all_columns = set(categorical_features + numerical_features)

# Exclude specific columns from being considered
excluded_columns = {'Unnamed: 0', 'isFraud', 'TransactionID'}

# Remove any columns from the dataframe that are not in the updated feature sets
removed_columns = []
for col in trIdTr.columns:
    if col not in all_columns and col not in excluded_columns:
        removed_columns.append(col)
        trIdTr.drop(col, axis=1, inplace=True)

# Print the names of removed columns
print("Removed columns:", removed_columns)

# Verify that all remaining columns are accounted for
remaining_columns = set(trIdTr.columns)
assert all(col in all_columns or col in excluded_columns for col in remaining_columns), "Some columns are not in either feature list!"

# Remove 'TransactionDT' and 'TransactionAmt' from numerical_features
numerical_features = [col for col in numerical_features if col not in ['TransactionDT', 'TransactionAmt']]

# Drop 'TransactionDT' and 'TransactionAmt' from the DataFrame
trIdTr.drop(['TransactionDT', 'TransactionAmt'], axis=1, inplace=True)

# create copy
trIdTr2 = trIdTr.copy()

## combining features: correlation with target

### evaluate which features are and aren't correlated with target isFraud


# Check correlation for numerical features with 'isFraud' (using Pearson's correlation)
numerical_corr = trIdTr[numerical_features].corrwith(trIdTr['isFraud']).sort_values(ascending=False)
print("Numerical Features Correlation with isFraud:")
print(numerical_corr)

# Plot a heatmap of numerical correlations with target for better visualization
plt.figure(figsize=(12, 6))
sns.heatmap(numerical_corr.to_frame(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation of Numerical Features with Target ('isFraud')")
plt.show()

## remove uncorrelated features (with isFraud)

# Set a threshold for weak correlation (e.g., below 0.1 or -0.1)
low_correlation_features = numerical_corr[abs(numerical_corr) < 0.1].index

# Remove features with low correlation
numerical_features = [feature for feature in numerical_features if feature not in low_correlation_features]

# Print features to be removed
print(f"Features with low correlation (removed): {low_correlation_features}")

## update trIdTr2 to include features only needed:

# Define the columns to include
included_columns = ['isFraud', 'Unnamed: 0', 'TransactionID'] + numerical_features + categorical_features

# Update trIdTr2 by selecting only the desired columns
trIdTr2 = trIdTr[included_columns]

# Print the shape of the updated dataframe
print(f"Updated dataframe shape: {trIdTr2.shape}")

#create copy
trIdTr3 = trIdTr2.copy()

# Identify duplicated columns
duplicate_columns = trIdTr3.columns[trIdTr3.columns.duplicated()]

# Log the duplicated columns
print(f"Duplicated columns: {duplicate_columns.tolist()}")

# Drop duplicate columns
trIdTr3 = trIdTr3.loc[:, ~trIdTr3.columns.duplicated()]

# train the lightgbm model

## set up parameter and train
### special note: early_stopping and log_evaluation should be callbacks 
### we use stratifiedKfold to reduce effect of imbalanced classsee 



# Prepare the features and target
X = trIdTr3.drop(columns=['isFraud', 'Unnamed: 0', 'TransactionID'])
y = trIdTr3['isFraud']

# Set up StratifiedKFold (5-fold)
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up LightGBM parameters
params = {
    'objective': 'binary',             # Binary classification
    'metric': 'auc',                   # Evaluation metric (AUC)
    'boosting_type': 'gbdt',           # Gradient Boosting Decision Tree
    'num_leaves': 31,                  # Number of leaves in one tree
    'learning_rate': 0.05,             # Learning rate
    'feature_fraction': 0.9,           # Fraction of features to be used in each iteration
    'bagging_fraction': 0.8,           # Fraction of data to be used in each iteration
    'bagging_freq': 5,                 # Frequency of bagging
    'verbose': -1                      # Suppress LightGBM output
}

# Early stopping and log evaluation callbacks
callbacks = [
    lgb.early_stopping(stopping_rounds=50),  # Stop if validation AUC does not improve for 50 rounds
    lgb.log_evaluation(period=10)            # Log the evaluation every 10 rounds
]

# Initialize variables to store the out-of-fold predictions and actual labels
oof_preds = np.zeros(X.shape[0])
roc_auc_scores = []

# Perform StratifiedKFold cross-validation
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"Training fold {fold+1}...")
    
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    # Convert to LightGBM dataset format
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val)

    # Train the model
    model = lgb.train(
        params,
        train_data,
        valid_sets=[train_data, val_data],
        valid_names=['train', 'valid'],
        num_boost_round=1000,                # Maximum number of boosting rounds
        callbacks=callbacks
    )

    # Make predictions on the validation set
    y_pred = model.predict(X_val, num_iteration=model.best_iteration)
    
    # Store the out-of-fold predictions
    oof_preds[val_idx] = y_pred
    
    # Calculate and print ROC AUC score for the fold
    roc_auc = roc_auc_score(y_val, y_pred)
    roc_auc_scores.append(roc_auc)
    print(f"Fold {fold+1} ROC AUC: {roc_auc:.4f}")

# Calculate and print the average ROC AUC across all folds
avg_roc_auc = np.mean(roc_auc_scores)
print(f"\nAverage ROC AUC across all folds: {avg_roc_auc:.4f}")

# Save the model to a file
model.save_model('safeBank_lightGBM_model.txt')



# predict here

